<a href="https://colab.research.google.com/github/Loreadad/Trabajo_final_ITBA/blob/main/Trabajo_Final_ITBA_Python_14_10_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import requests
import sqlite3 as sql
import datetime
import requests as r
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from optparse import Values
from tkinter.tix import Tree
import json



# Acceso al Menú Principal
def home():
    menu_principal = input('Seleccione una opción: \n1) Actualización de datos\n2) Visualización de datos\n')

#1) Actualización de datos
    if menu_principal == '1':

    # Se piden los parametros de ticker, fecha inicio y fecha fin
      ticker = input('\nIngrese ticker a pedir:').upper()
      fecha_inicio = input("Ingrese fecha de inicio (YYYY-MM-DD): ")
      año, mes, dia = map(int, fecha_inicio.split('-'))
      fecha_fin = input("Ingrese fecha de inicio (YYYY-MM-DD): ")
      año, mes, dia = map(int, fecha_fin.split('-'))


      #  Se realiza la conexión a la API
      try:
          print('\nPidiendo datos ...')
          ticker_get = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{fecha_inicio}/{fecha_fin}?adjusted=true&sort=asc&limit=120&apiKey=G_Yb5HG6ldlS0k0SHQdr8w6Q4FKYJEVE')
          objeto_api = ticker_get.json()
      except:
          print('No se puede conectar a la API. Por favor, reintente nuevamente.')    

      #  Almacenamiento en la base de datos
      if objeto_api['resultsCount'] != 0:

          try:
              conexion = sql.connect('trabajo_final.db')
              cursor = conexion.cursor()
          except:
              print('No se puede conectar a la base de datos. Por favor, reintente nuevamente.')

          for x in range(len(objeto_api['results'])):
              valor_cierre = objeto_api['results'][x]['c']

              cursor.execute(f'''INSERT INTO api_polygon (ticker, fecha_inicio, fecha_fin, cierre) VALUES ('{ticker}', '{fecha_inicio}', '{fecha_fin}', '{valor_cierre}');''')
              conexion.commit()
          
          cursor.close()

          print(f'\nDatos guardados correctamente')

      else:
          print('\nNo se pudieron guardar los datos en la base. Por favor, reintente nuevamente.')

#2) Visualización de datos
    elif menu_principal == '2':

        opcion_submenu = input('Seleccione una opción: \n1) Resumen\n2) Gráfico de ticker\n')

    #1) Resumen
        if opcion_submenu == '1':

            try:
                conexion = sql.connect('trabajo_final.db')
                cursor = conexion.cursor()
                resumen_tabla = cursor.execute('''SELECT id, ticker, fecha_inicio, fecha_fin FROM api_polygon;''').fetchall()

                print('\nLos tickers guardados en la base de datos son:')

                for x in range (len(resumen_tabla)):
                    print(f'{x+1}. {resumen_tabla[x][1]} - {resumen_tabla[x][2]} <-> {resumen_tabla[x][3]}')

            except:
                print('No se pudo obtener los datos solicitados. Por favor, reintente nuevamente.')
            finally:
                conexion.close()

    #2) Gráfico de ticker
        if opcion_submenu == '2':

               ticker_grafico = input('\nIngrese el ticker a graficar:').upper()
               startTime= input("Ingrese una fecha de inicio con el siguiente formato (YYYY-MM-DD): ")
               año, mes, dia = map(int, startTime.split('-'))
               endtime= input("Ingrese una fecha de cierre con el siguiente formato (YYYY-MM-DD): ")
               año, mes, dia = map(int, endtime.split('-'))
               fecha = startTime + "/" + endtime
               url = "https://api.polygon.io/v2/aggs/ticker/"+ str(ticker_grafico) +"/range/1/day/"+ str(fecha)+ "?adjusted=true&sort=asc&limit=120&apiKey=G_Yb5HG6ldlS0k0SHQdr8w6Q4FKYJEVE"
               payload={}
               files={}
               headers = {}
               response = r.request("GET", url, headers=headers, data=payload, files=files)
               res = []
               data = response.json()
               for s in data['results']:
                        res.append(s)
                        df = pd.DataFrame(res)
                        df ['Date'] = pd.to_datetime (df['t'], unit ='ms')
                        df = df.rename(columns={
                            'o': 'open',
                            'c': 'close',
                            'h': 'high',
                            'l': 'low',
                            'v': 'volume',
                            'n': 'transactions'})
                        conn = sql.connect ('df.db')
                        df.to_sql ('df', conn, if_exists='replace')

                        conn = sql.connect ('df.db')
                        df = pd.read_sql ('SELECT * FROM df', conn)

             # Pedido al servidor
               def pedido_servidor(ticket,date_from,date_to):
                    results = []
                    url = "https://api.polygon.io/v2/aggs/ticker/"
                    stocksT = ticket
                    mult = '1'
                    timestamp = 'day'
                    adjusted = 'adjusted=true'
                    sort = 'sort=asc'
                    limit = '1000'
                    apiKey = 'apiKey=G_Yb5HG6ldlS0k0SHQdr8w6Q4FKYJEVE'
                    x = r.get(url+stocksT+'/range/'+mult+ '/'+timestamp+'/'+ date_from+'/'+ date_to +'?adjusted=true&sort=asc&limit=120&apiKey=G_Yb5HG6ldlS0k0SHQdr8w6Q4FKYJEVE')
                    y =x.json()
                    date = 0
                    if str(x.status_code)[0] == '4':
                        print('Hubo un Error')
                    else:
                        if y['queryCount'] == 0: print('Sin Valor')
                        else: results = y ['results']
               def check_ticker(name):
                    pars = r.get("https://api.polygon.io/v2/aggs/ticker/"+name+'/range/1/day/2021-07-22/2021-07-22?adjusted=true&sort=asc&limit=120&apiKey=G_Yb5HG6ldlS0k0SHQdr8w6Q4FKYJEVE')
                    complete_name = pars.json()['results']
                    return complete_name
                    
              #print(response.text)
               fig = go.Figure()
               fig.add_trace(go.Candlestick(x=df.index,
                              open=df['open'],
                              high=df['high'],
                              low=df['low'],
                              close=df['close'], name = 'market data'))
               fig.update_layout(
                          title='Evolución Precio',
                          yaxis_title='Precio (USD)')
               fig.update_xaxes(
                        rangeslider_visible=True,
                        rangeselector=dict(
                            buttons=list([
                                dict(count=15, label="15m", step="minute", stepmode="backward"),
                                dict(count=45, label="45m", step="minute", stepmode="backward"),
                                dict(count=1, label="HTD", step="hour", stepmode="todate"),
                                dict(count=3, label="3h", step="hour", stepmode="backward"),
                                dict(step="all")
                            ])
                        )
                    )
               fig.show()
        else: print ('\n Seleccione una opción válida')
        #home()

decision = True

while decision:
    home()

Seleccione una opción: 
1) Actualización de datos
2) Visualización de datos
2
Seleccione una opción: 
1) Resumen
2) Gráfico de ticker
2

Ingrese el ticker a graficar:TSLA
Ingrese una fecha de inicio con el siguiente formato (YYYY-MM-DD): 2021-01-01
Ingrese una fecha de cierre con el siguiente formato (YYYY-MM-DD): 2021-02-01


KeyboardInterrupt: ignored